#### __ARGs-OAP__

Online Analysis Pipeline for Antibiotic Resistance Genes Detection from Metagenomic Data Using an Integrated Structured ARG Database 

* __Github repo__

https://github.com/biofuture/Ublastx_stageone

https://github.com/xinehc/args_oap 

* __Singularity container__

https://ndownloader.figshare.com/files/25953791

__Notas:__

- Al parecer el programa no reconoce los enlaces simbolicos ni el almacenamiento del disco grupal (quizas es el contenedor). Hay que probar ejecutar en el directorio /home 

In [1]:
import polars as pl

import os
import shutil

DIR = "/srv/storage/oceania@storage2.lille.grid5000.f/npilquinao/BEM-ISP/argsoap"

In [4]:
!args_oap stage_one -h

usage: args_oap stage_one [-h] -i DIR -o DIR [-t INT] [-f EXT] [--e1 FLOAT]
                          [--e2 FLOAT] [--id FLOAT] [--qcov FLOAT] [--skip]
                          [--keep] [--database FILE]

options:
  -h, --help            show this help message and exit

required arguments:
  -i DIR, --indir DIR   Input folder.
  -o DIR, --outdir DIR  Output folder.

optional arguments:
  -t INT, --thread INT  Number of threads. [8]
  -f EXT, --format EXT  File format in input folder (--indir), gzipped (*.gz)
                        optional. [fq]
  --e1 FLOAT            E-value cutoff for GreenGenes. [1e-10]
  --e2 FLOAT            E-value cutoff for Essential Single Copy Marker Genes.
                        [3]
  --id FLOAT            Identity cutoff (in percentage) for Essential Single
                        Copy Marker Genes. [45]
  --qcov FLOAT          Query cover cutoff (in percentage) for Essential
                        Single Copy Marker Genes. [0]
  --skip                

In [1]:
!args_oap stage_two -h

usage: args_oap stage_two [-h] -i DIR [-t INT] [-o DIR] [--e FLOAT]
                          [--id FLOAT] [--qcov FLOAT] [--length INT]
                          [--blastout FILE] [--database FILE]
                          [--structure1 FILE] [--structure2 FILE]
                          [--structure3 FILE]

options:
  -h, --help            show this help message and exit

required arguments:
  -i DIR, --indir DIR   Input folder. Should be the output folder of stage_one
                        (containing <metadata.txt> and <extracted.fa>).

optional arguments:
  -t INT, --thread INT  Number of threads. [8]
  -o DIR, --outdir DIR  Output folder, if not given then same as input folder
                        (--indir). [None]
  --e FLOAT             E-value cutoff for target sequences. [1e-7]
  --id FLOAT            Identity cutoff (in percentage) for target sequences.
                        [80]
  --qcov FLOAT          Query cover cutoff (in percentage) for target
                  

In [6]:
%%bash

args_oap stage_one \
    --indir /home/npilquinao/BEM-ISP/group_storage/argsoap/samples\
    --outdir /home/npilquinao/BEM-ISP/group_storage/argsoap/stage_one\
    --thread 2\
    --format fq.gz

[2024-05-16 06:20:31] INFO: Building databases ...
[2024-05-16 06:20:37] INFO: Processing </home/npilquinao/BEM-ISP/group_storage/argsoap/samples/VA1046_1.fq.gz> (1/24) ...
[2024-05-16 06:28:17] INFO: Processing </home/npilquinao/BEM-ISP/group_storage/argsoap/samples/VA1046_2.fq.gz> (2/24) ...


[2024-05-16 06:35:56] INFO: Processing </home/npilquinao/BEM-ISP/group_storage/argsoap/samples/VA1101_1.fq.gz> (3/24) ...


### __MALO NO USAR__

__Get container__

```bash
wget -O $workpath/argoapv2.5.sif https://ndownloader.figshare.com/files/25953791

##### __Metadata & data format__

__Samples (.fq format)__
```bash
/home/npilquinao/BEM-ISP/group_storage/argsoap/
```
__Metadata file (.txt)__
```bash
/home/npilquinao/BEM-ISP/group_storage/argsoap/metadata.txt

In [3]:
metadata = pl.DataFrame({
    'SampleID': list(range(1, 13)),
    'Name': ['VA61','VA418','VA585','VA692','VA1046','VA1101','VA1184','VA1565','VA1788','VA2067','VA2464','VA2588'],
    'Category': ['2_carbapenemase' for _ in range(12)]
})
metadata.write_csv(f"{workpath}/metadata.tsv", separator='\t')

In [4]:
for dirpath, dirnames, filenames in os.walk(samples):
    for filename in filenames:
        if filename.endswith('.fastq.gz'):
            parent_dir_name = os.path.basename(dirpath)
            new_filename_base = parent_dir_name.split('_')[0]

            if '_R1' in filename:
                new_filename = new_filename_base + '_1.fq.gz'
            elif '_R2' in filename:
                new_filename = new_filename_base + '_2.fq.gz'
            else:
                continue

            shutil.copy(os.path.join(dirpath, filename), os.path.join(workpath, "argsoap_samples", new_filename))

##### __ARGs-OAP__
```bash
* The inputdir follow by the -B is the directory you put your fastq files
* -i Input files directory, required
* -m meta data file, required
* -o Output files directory, default current directory
* -z whether the fq files were .gz format, default(none)
* -n number of threads used, default 1

In [5]:
%%bash

cd /srv/storage/oceania@storage2.lille.grid5000.f/npilquinao/BEM-ISP/argsoap

singularity exec \
    -B /srv/storage/oceania@storage2.lille.grid5000.f/npilquinao/BEM-ISP/argsoap \
    argoapv2.5.sif /home/argsoapv2.5/argoap_version2.5 \
    -i argsoap_samples \
    -m metadata.txt \
    -o output \
    -z \
    -n 60

INFO:    Converting SIF file to temporary sandbox...


perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = "en_US:en",
	LC_ALL = "en_US.UTF-8",
	LANG = "en_US.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to the standard locale ("C").
No such file or directory
INFO:    Cleaning up image...


CalledProcessError: Command 'b'\ncd /srv/storage/oceania@storage2.lille.grid5000.f/npilquinao/BEM-ISP/argsoap\n\nsingularity exec \\\n    -B /srv/storage/oceania@storage2.lille.grid5000.f/npilquinao/BEM-ISP/argsoap \\\n    argoapv2.5.sif /home/argsoapv2.5/argoap_version2.5 \\\n    -i argsoap_samples \\\n    -m metadata.txt \\\n    -o output \\\n    -z \\\n    -n 60\n'' returned non-zero exit status 2.